# Decision Trees

In [ ]:
import numpy as np

* Nodes: decision nodes
* Edges: values
* Leafs: output / decision / answer

### ID3: top-down algorithm for inducing a decision tree
- Loop:
    - Pick the best attribute A to split on
    - Assign A as decision node
    - Assign examples to leafs for each value
    - Stop if stopping criterion is met
    
### Expressiveness
?

### Inductive bias
- Restriction bias
- Preference bias
    - Prefers good splits early on
    - Prefers correct trees
    - Prefers shorter trees (a consequence of the 1st preference)

### Entropy
$$-\sum_{v}p(v)logp(v)$$

In [71]:
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    p = counts / np.sum(counts)
    return -np.sum(p * np.log2(p))

def entropy_binary(x):
    trues = np.sum(x)
    percentage_true = trues / len(x)
    if percentage_true == 1:
        return 1
    else:
        percentage_false = 1 - percentage_true
        p = np.array([percentage_true, percentage_false])
        return -np.sum(p * np.log2(p))

In [72]:
x1 = [True, False, False, False]
x2 = [True, False]
x3 = [True, True, True]
x4 = [True] * 99 + [False]

In [73]:
entropy(x1)

0.81127812445913283

In [74]:
entropy(x2)

1.0

In [75]:
entropy(x4)

0.080793135895911181

In [76]:
entropy(x3)

-0.0

In [77]:
entropy_binary(x1)

0.81127812445913283

In [78]:
entropy_binary(x2)

1.0

In [79]:
entropy_binary(x3)

1

In [80]:
# TODO: not exactly the same
entropy_binary(x4)

0.080793135895911236

### Information gain
$$Gain(S, A)=Entropy(S)-\sum_{v}\frac{|S_v|}{|S|}Entropy(S_v)$$

In [ ]:
def information_gain(s, a):
    return entropy(s) - sum(
        [ for ?]
    )

### References
- https://www.udacity.com/course/machine-learning--ud262